In [ ]:
#Random Forest W/ Hyperparameter Tuning Checked
import numpy as np
import pandas as pd
import datetime
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix,roc_auc_score,f1_score,balanced_accuracy_score
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

In [ ]:
def tn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0]
def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1]
def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0]
def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 1]

In [ ]:
def grid_search_wrapper(refit_score):
    """
    fits a GridSearchCV classifier using refit_score for optimization
    """
    
    print('\nRefit by ',refit_score)
    
    #start model
    start_time  = datetime.datetime.now()
    print('\nStart time',start_time)
    
    skf = StratifiedKFold(n_splits=10)
    grid_search = GridSearchCV(clf, param_grid, scoring=scorers, refit=refit_score,cv=skf, return_train_score=True, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    #end model fit
    end_time = datetime.datetime.now()
    print('\nEnd time',end_time)

    return grid_search

In [ ]:
#imbalanced dataset
#print('imbalanced dataset')
#DATA_PATH='data_vec_non_alpha_numericBy_FeatSelect.csv'

In [ ]:
#balanced dataset
print('balanced dataset')
DATA_PATH='data_vec_non_alpha_numericBy_FeatSelect5000.csv'

In [ ]:
dataset=pd.read_csv(DATA_PATH)
y=dataset['Result']

In [ ]:
#W/O Entropy
#print('W/O Entropy')
#X=dataset[['IP_Address','EXE','Sensitive_Word','double_slash_redirecting','W3_HTTP_token','who_is','dash_sign','at_sign','dot_sign','free_host','port']]
#W/ Entropy
print('W/ Entropy')
X=dataset.drop('Result',axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
# show the distribution
print('y_train class distribution\n')
print(y_train.value_counts(normalize=True))
print('\ny_test class distribution')
print('\n',y_test.value_counts(normalize=True))

In [ ]:
clf = RandomForestClassifier()

In [ ]:
param_grid = {
    'n_estimators' : [100,110,120,130,140,150,160,170,180,190,200,300,400,500,600,700],
    #'n_estimators' : [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,300,400,500,600,700],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False]
}

In [ ]:
scorers = {
    'precision_micro_score': make_scorer(precision_score, average='micro'),
    'precision_score': make_scorer(precision_score, average='weighted'),
    'recall_micro_score': make_scorer(recall_score, average='micro'),
    'recall_score': make_scorer(recall_score, average='weighted'),
    'accuracy_score': make_scorer(accuracy_score),
    'roc_score_scorer' : make_scorer(roc_auc_score,average='weighted'),
    'metrics_roc_auc_score' : 'roc_auc',
    'f1_score' : make_scorer(f1_score, average='weighted'),
    'tp': make_scorer(tp),
    'tn': make_scorer(tn),
    'fp': make_scorer(fp),
    'fn': make_scorer(fn)
}

In [ ]:
refit_scorer='metrics_roc_auc_score'
print('Refit by ',refit_scorer)
grid_search_clf = grid_search_wrapper(refit_score=refit_scorer)

In [ ]:
# make the predictions
y_pred = grid_search_clf.predict(X_test)

In [ ]:
print('\nBest params for \n',grid_search_clf.best_params_)
best_result = grid_search_clf.best_score_
print('\nBest Result :',best_result)

In [ ]:
# confusion matrix on the test data.
print('\nConfusion matrix of Random Forest optimized for the test data:')
print('\n',str(pd.DataFrame(confusion_matrix(y_test, y_pred), columns=['pred_neg', 'pred_pos'], index=['neg', 'pos'])))
results = pd.DataFrame(grid_search_clf.cv_results_)
results = results.sort_values(by='mean_test_metrics_roc_auc_score', ascending=False)